Testing ConvNet on dummy images

In [1]:
# import needed stuff
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import time

# verify tensorflow installation
print("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.0.0


In [2]:
# load image files
DATADIR = "C:/Users/gabriel.l.santiago/Documents/PetImages"
CATEGORIES = ["Dog", "Cat"]
MODEL_NAME = "Cats-vs-dogs-128x2-3conv-Dense0"

In [3]:
# resize images
IMG_SIZE = 50

In [4]:
# create training set
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) # path to pics
        class_num = CATEGORIES.index(category)
        
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_training_data()
print(len(training_data))

24946


In [5]:
# randomize training data
random.shuffle(training_data)

for sample in training_data[:10]:
    print(sample[1])

# prepare data for modeling
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

1
1
0
0
0
1
1
0
1
1


In [6]:
# save data
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [7]:
# load data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

In [8]:
# normalize data
X = X/255.0

# number of features
layer_size = 128

# setup model
model = Sequential()
model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:])) # conv layer
model.add(Activation("relu")) # activation
model.add(MaxPooling2D(pool_size=(2,2))) # pooling layer

model.add(Conv2D(layer_size, (3,3))) # another conv layer
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(layer_size, (3,3))) # another conv layer
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # flatten layers from 1D -> 2D

# dense layer
# model.add(Dense(layer_size))
# model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation("sigmoid"))

In [9]:
# add tensorboard as keras callback
NAME = MODEL_NAME, "-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir="logs\{}".format(NAME))

In [10]:
# train model
# launch tensorboard: tensorboard --logdir logs
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=20, validation_split=0.3, callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/20
17462/17462 [==============================] - 101s 6ms/sample - loss: 0.6480 - accuracy: 0.6167 - val_loss: 0.5827 - val_accuracy: 0.7098
Epoch 2/20
17462/17462 [==============================] - 574s 33ms/sample - loss: 0.5458 - accuracy: 0.7244 - val_loss: 0.5006 - val_accuracy: 0.7568
Epoch 3/20
17462/17462 [==============================] - 96s 6ms/sample - loss: 0.4791 - accuracy: 0.7692 - val_loss: 0.4629 - val_accuracy: 0.7855
Epoch 4/20
17462/17462 [==============================] - 98s 6ms/sample - loss: 0.4356 - accuracy: 0.7940 - val_loss: 0.4705 - val_accuracy: 0.7850
Epoch 5/20
17462/17462 [==============================] - 102s 6ms/sample - loss: 0.3926 - accuracy: 0.8237 - val_loss: 0.4350 - val_accuracy: 0.8029
Epoch 6/20
17462/17462 [==============================] - 100s 6ms/sample - loss: 0.3497 - accuracy: 0.8435 - val_loss: 0.4199 - val_accuracy: 0.8092
Epoch 7/20
17462/17462 [============================

In [11]:
# save model 
model.save(MODEL_NAME, ".model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Cats-vs-dogs-128x2-3conv-Dense0\assets


In [12]:
# use model on images
def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    new_array = np.array(new_array).astype(np.float32)
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [47]:
# load model
# model = tf.keras.models.load_model("Cats-vs-dogs-64x2-3conv-Dense1.model")
model = tf.keras.models.load_model("Cats-vs-dogs-64x2-3conv-Dense0")
# model = tf.keras.models.load_model("Cats-vs-dogs-128x2-3conv-Dense0")

In [52]:
# make prediction
# ALWAYS pass a list even if one prediction only
prediction = model.predict([prepare('cat1.jpg')])
print(CATEGORIES[int(prediction[0][0])])

Dog
